In [3]:
from Analysis import *
import Analysis
from tssb.utils import load_time_series_segmentation_datasets
from tssb.evaluation import covering
import time
import pandas as pd

In [4]:
datasets = ["ArrowHead", "InlineSkate", "Plane"]
results = []
tssb = load_time_series_segmentation_datasets(names=datasets)

# Test ClaSP implementation
for _, (ts_name, window_size, cps, ts) in tssb.iterrows():
    start_time = time.time()
    found_cps = BinaryClaSPSegmentation().fit_predict(ts)
    end_time = time.time()
    score = covering({0: cps}, found_cps, ts.shape[0])
    results.append({
        'Algo': 'ClaSP',
        'Time Series': ts_name,
        'True Change Points': cps,
        'Found Change Points': found_cps.tolist(),
        'Score': score,
        'Run Time': (end_time - start_time)
    })

# Test HNSW implementation
for _, (ts_name, window_size, cps, ts) in tssb.iterrows():
    start_time = time.time()
    found_cps = AltBinaryClaSPSegmentation(knn=AltKNN(method="hnsw")).fit_predict(ts)
    end_time = time.time()
    score = covering({0: cps}, found_cps, ts.shape[0])
    results.append({
        'Algo': 'HNSW',
        'Time Series': ts_name,
        'True Change Points': cps,
        'Found Change Points': found_cps.tolist(),
        'Score': score,
        'Run Time': (end_time - start_time)
    })

# Test FJLT implementation
for _, (ts_name, window_size, cps, ts) in tssb.iterrows():
    start_time = time.time()
    found_cps = AltBinaryClaSPSegmentation(knn=AltKNN(method="fjlt")).fit_predict(ts)
    end_time = time.time()
    score = covering({0: cps}, found_cps, ts.shape[0])
    results.append({
        'Algo': 'FJLT',
        'Time Series': ts_name,
        'True Change Points': cps,
        'Found Change Points': found_cps.tolist(),
        'Score': score,
        'Run Time': (end_time - start_time)
    })

# Create DataFrame and display results
results_df = pd.DataFrame(results)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("\n=== Full Results DataFrame ===")
print(results_df.to_string())
print("\n=== Pivoted View ===")
print(results_df.pivot(index='Algo', columns='Time Series').to_string())

ValueError: euclidean is not a valid distance. Implementations include: znormed_euclidean_distance, euclidean_distance, cinvariant_euclidean_distance

In [ ]:
# Display a cleaner view focusing on scores and run times
print("\n=== Performance Comparison ===")
print(results_df.drop(['True Change Points', 'Found Change Points'], axis='columns')
      .pivot(index='Algo', columns='Time Series').to_string())